# Hazard assessment for river flooding using river discharge statistics: Accessing data

- A workflow from the CLIMAAX [Handbook](https://handbook.climaax.eu/) and [FLOODS](https://github.com/CLIMAAX/FLOODS) GitHub repository.
- See our [how to use risk workflows](https://handbook.climaax.eu/notebooks/workflows_how_to.html) page for information on how to run this notebook.

This notebook illustrates how the river discharges dataset can be downloaded via API from the Copernicus Data Store (CDS) for subsequent use in the analysis.

:::{note}
The CDS dataset is downloaded for the entire Europe, it is not possible to subset it by area prior to downloading.
CLIMAAX provides a [dataset mirror](https://handbook.climaax.eu/resources/datasets/river_discharges_mirror.html) for regional data access.
To use the dataset mirror, skip this data access notebook and go directly to the [timeseries analysis](./hazard_assessment_timeseries_analysis.ipynb) notebook.
:::

## Preparation work

### Load libraries

`````{admonition} Find more info about the libraries used in this workflow here
:class: hint dropdown

In this notebook we will use the following Python libraries:
- [os](https://docs.python.org/3/library/os.html) - provides a way to interact with the operating system, allowing the creation of directories and file manipulation.
- [zipfile](https://docs.python.org/3/library/zipfile.html) - library for working with ZIP archives.
- [cdsapi](https://cds.climate.copernicus.eu/how-to-api) - a library to request data from the datasets listed in the CDS catalogue.
- [xarray](https://docs.xarray.dev/en/stable/) - library for working with labelled multi-dimensional arrays.

These libraries enable the download and preprocessing of the datasets included in the river discharge analysis workflow.
`````

In [21]:
import os
import zipfile

import cdsapi
import xarray as xr

### Create the directory structure
In the next cell will create the directory called `FLOOD_RIVER_discharges` in the same directory where this notebook is saved. A folder for storing data will be made as well.

In [22]:
# Define the folder for the flood workflow
workflow_folder = 'FLOOD_RIVER_discharges'
os.makedirs(workflow_folder, exist_ok=True)

data_folder = os.path.join(workflow_folder, 'data')
os.makedirs(data_folder, exist_ok=True)

data_folder_catch = os.path.join(data_folder, 'EHYPEcatch')
os.makedirs(data_folder_catch, exist_ok=True)

## Data access parameters

In the cell below we will select three GCM-RCM model combinations (see dataset documentation for the available combinations). Using several model combinations helps to assess the uncertainty range due to the different climate models in the river discharges data.  

In [23]:
gcms = ["ec_earth", "hadgem2_es", "mpi_esm_lr", "ec_earth", "mpi_esm_lr", "hadgem2_es"]
rcms = ["cclm4_8_17", "racmo22e", "rca4", "racmo22e", "csc_remo2009", "rca4"]
ens_members = ['r12i1p1', 'r1i1p1', 'r1i1p1', 'r12i1p1', 'r1i1p1', 'r1i1p1']

hydrological_models = [
    "e_hypecatch_m00", "e_hypecatch_m01", "e_hypecatch_m02", "e_hypecatch_m03",
    "e_hypecatch_m04", "e_hypecatch_m05", "e_hypecatch_m06", "e_hypecatch_m07"
]

We also need to initialize the API client to be able to make connection to the CDS servers for downloading the data.

In [ ]:
client = cdsapi.Client()

## Downloading river discharge timeseries (historical daily values)

First, we will download catchment-level discharge data for the historical period. Data is available based on different E-HYPEcatch model realizations. We will download all model realizations.

The daily timeseries are downloaded for the period of 1991-2005. The total duration of 15 years is chosen as a minimum time period for deriving representative long-term statistics. If a different period is required for comparing to local observations, the selection can be adjusted below as part of the API request under "period".

The total size of the preconfigured data request is about 36 GB.

In [ ]:
for gcm, rcm, ens_member in zip(gcms, rcms, ens_members):
    file = os.path.join(data_folder_catch, 'download.zip')
    dataset = "sis-hydrology-variables-derived-projections"
    request = {
        "product_type": "essential_climate_variables",
        "variable": ["river_discharge"],
        "variable_type": "absolute_values",
        "time_aggregation": "daily",
        "experiment": ["historical"],
        "hydrological_model": hydrological_models,
        "rcm": rcm,
        "gcm": gcm,
        "ensemble_member": ens_member,
        "period": ["1991_2000", "2001_2005"]
    }
    client.retrieve(dataset, request, file)

    # Unzip the file that was just downloaded, and remove the zip file
    with zipfile.ZipFile(file, 'r') as zObject:
        zObject.extractall(path=data_folder_catch)
    os.remove(file)

In [24]:
def preprocess_daily(ds):
    filename = os.path.basename(ds.encoding['source'])
    _, _, catchmodel, gcm, _, _, rcm, _, _, _ = filename.split('_')
    return ds.expand_dims({
        'catchmodel': [catchmodel],
        'gcm_rcm': [f'{gcm}_{rcm}'],
    })

In [25]:
ds_day = xr.open_mfdataset(
    os.path.join(data_folder_catch, 'rdis_day_E-HYPEcatch*-EUR-11_*_catch_v1.nc'),
    preprocess=preprocess_daily,
    chunks="auto"
)

In [26]:
ds_day

<xarray.Dataset> Size: 37GB
Dimensions:     (catchmodel: 8, gcm_rcm: 6, time: 5479, id: 34810)
Coordinates:
  * catchmodel  (catchmodel) object 64B 'E-HYPEcatch00-EUR-11' ... 'E-HYPEcat...
  * gcm_rcm     (gcm_rcm) object 48B 'ICHEC-EC-EARTH_CLMcom-CCLM4-8-17' ... '...
  * time        (time) datetime64[ns] 44kB 1991-01-01 1991-01-02 ... 2005-12-31
  * id          (id) int32 139kB 8801654 8000123 8212459 ... 9605711 9601936
Data variables:
    rdis        (catchmodel, gcm_rcm, time, id) float32 37GB dask.array<chunksize=(1, 1, 963, 34810), meta=np.ndarray>
Attributes: (12/26)
    CDI:                      Climate Data Interface version 1.9.5 (http://mp...
    CDO:                      Climate Data Operators version 1.9.5 (http://mp...
    source:                   A set of EURO-CORDEX EUR-11 RCMS were bias adju...
    institution:              SMHI, www.smhi.se
    Conventions:              CF-1.6
    NCO:                      4.4.8
    ...                       ...
    invar_experiment_name:    rcp45
    invar_rcm_model_id:       CLMcom-CCLM4-8-17
    history:                  
    time_period:              1991-2000
    contact:                  copernicus-support@ecmwf.int
    data_quality:             C3S_424_SMHI uses state of the art, quality con...

In [27]:
ds_day.to_netcdf(os.path.join(data_folder, 'rdis_day_E-HYPEcatch_allmodels.nc'))

## Downloading river discharge timeseries (monthly means)

Next we will download the historical monthly means of river discharges for 1971-2000 from the E-HYPEcatch models which are useful for checking longer-term statistics of river discharges in the historical climate.

The total size of the preconfigured dataset is about 1 GB.

In [ ]:
for gcm, rcm, ens_member in zip(gcms, rcms, ens_members):
    file = os.path.join(data_folder_catch, 'download.zip')
    dataset = "sis-hydrology-variables-derived-projections"
    request = {
        "product_type": "climate_impact_indicators",
        "variable": ["river_discharge"],
        "variable_type": "absolute_values",
        "time_aggregation": "monthly_mean",
        "experiment": ["historical"],
        "hydrological_model": hydrological_models,
        "rcm": rcm,
        "gcm": gcm,
        "ensemble_member": ens_member,
        "period": ["1971_2000"]
    }
    client.retrieve(dataset, request, file)

    # Unzip the file that was just downloaded, and remove the zip file
    with zipfile.ZipFile(file, 'r') as zObject:
        zObject.extractall(path=data_folder_catch)
    os.remove(file)


We will download monthly means of discharges for future periods of 2011-2040, 2041-2070 and 2071-2100:

In [ ]:
for gcm, rcm, ens_member in zip(gcms, rcms, ens_members):
    for period in ["2011_2040", "2041_2070", "2071_2100"]:
        file = os.path.join(data_folder_catch, 'download.zip')
        dataset = "sis-hydrology-variables-derived-projections"
        request = {
            "product_type": "climate_impact_indicators",
            "variable": ["river_discharge"],
            "variable_type": "absolute_values",
            "time_aggregation": "monthly_mean",
            "experiment": ["rcp_4_5","rcp_8_5"],
            "hydrological_model": hydrological_models,
            "rcm": rcm,
            "gcm": gcm,
            "ensemble_member": ens_member,
            "period": period
        }
        client.retrieve(dataset, request, file)

        # Unzip the file that was just downloaded, and remove the zip file
        with zipfile.ZipFile(file, 'r') as zObject:
            zObject.extractall(path=data_folder_catch)
        os.remove(file)

We will make use of a preprocessing function to write model names and scenarios to the dataset dimensions:

In [9]:
def preprocess_monthly_mean(ds):
    filename = os.path.basename(ds.encoding['source'])
    _, _, _, catchmodel, gcm, scenario, _, rcm, _, time_period, _, _ = filename.split('_')
    ds['time'] = ds.time.dt.month
    return ds.expand_dims({
        'time_period': [time_period],
        'scenario': [scenario],
        'catchmodel': [catchmodel],
        'gcm_rcm': [f'{gcm}_{rcm}'],
    })

Now we can read the dataset of monthly means of river discharges into a single dataset variable and save it on disk as one file for ease of future access.

In [10]:
ds_monmean = xr.open_mfdataset(
    os.path.join(data_folder_catch, 'rdis_ymonmean_abs_E-HYPEcatch*-EUR-11_*_na_*_catch_v1.nc'),
    preprocess=preprocess_monthly_mean,
    chunks='auto'
)

In [11]:
ds_monmean

<xarray.Dataset> Size: 963MB
Dimensions:        (time_period: 4, scenario: 3, catchmodel: 8, gcm_rcm: 6,
                    time: 12, id: 34810)
Coordinates:
  * time_period    (time_period) object 32B '1971-2000' ... '2071-2100'
  * catchmodel     (catchmodel) object 64B 'E-HYPEcatch00-EUR-11' ... 'E-HYPE...
  * gcm_rcm        (gcm_rcm) object 48B 'ICHEC-EC-EARTH_CLMcom-CCLM4-8-17-v1'...
  * time           (time) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
  * id             (id) int32 139kB 8801654 8000123 8212459 ... 9605711 9601936
  * scenario       (scenario) object 24B 'historical' 'rcp45' 'rcp85'
Data variables:
    rdis_ymonmean  (time_period, scenario, catchmodel, gcm_rcm, time, id) float32 962MB dask.array<chunksize=(2, 1, 1, 1, 12, 34810), meta=np.ndarray>
Attributes: (12/28)
    CDI:                      Climate Data Interface version 1.9.5 (http://mp...
    Conventions:              CF-1.6
    NCO:                      netCDF Operators version 4.7.7 (Homepage = http...
    comment:                  -
    CDO:                      Climate Data Operators version 1.9.5 (http://mp...
    history:                  CDO commands (last cdo command first and separa...
    ...                       ...
    invar_hm_model_id:        Hydrological models in the order of the variabl...
    invar_experiment_name:    rcp45
    time_coverage_start:      19710101
    time_coverage_end:        20001231
    variable_name:            rdis_ymonmean
    contact:                  copernicus-support@ecmwf.int

In [14]:
ds_monmean.to_netcdf(os.path.join(data_folder, 'rdis_ymonmean_abs_E-HYPEcatch_allmodels.nc'))

## Downloading data on flood occurence (extreme river discharges)

We will download river discharge data corresponding to the 10-year and 50-year return periods (extreme river discharges projected to be exceeded once in 10 years and once in 50 years). Similarly to the timeseries data, we will download this data for different climate scenarios, timelines and catchment models.

Downloading 10-year and 50-year return period river discharges for the historical climate:

In [ ]:
for gcm, rcm, ens_member in zip(gcms, rcms, ens_members):
    file = os.path.join(data_folder_catch, 'download.zip')
    dataset = "sis-hydrology-variables-derived-projections"
    request = {
        "product_type": "climate_impact_indicators",
        "variable": ["flood_recurrence_10_years_return_period",
                     "flood_recurrence_50_years_return_period"],
        "variable_type": "absolute_values",
        "time_aggregation": "annual_mean",
        "experiment": ["historical"],
        "hydrological_model": hydrological_models,
        "rcm": rcm,
        "gcm": gcm,
        "ensemble_member": ens_member,
        "period": ["1971_2000"]
    }   
    client.retrieve(dataset, request, file)

    # Unzip the file that was just downloaded, and remove the zip file
    with zipfile.ZipFile(file, 'r') as zObject:
        zObject.extractall(path=data_folder_catch)
    os.remove(file)

Downloading 10-year and 50-year return period river discharges for the future time periods in terms of absolute and relative values:

In [ ]:
for variable_type in ["absolute_values", "relative_change_from_reference_period"]:
    for gcm, rcm, ens_member in zip(gcms, rcms, ens_members):
        for period in ["2011_2040", "2041_2070", "2071_2100"]:
                file = os.path.join(data_folder_catch, 'download.zip')
                dataset = "sis-hydrology-variables-derived-projections"
                request = {
                    "product_type": "climate_impact_indicators",
                    "variable": ["flood_recurrence_10_years_return_period",
                                 "flood_recurrence_50_years_return_period"],
                    "variable_type": variable_type,
                    "time_aggregation": "annual_mean",
                    "experiment": ["rcp_4_5", "rcp_8_5"],
                    "hydrological_model": hydrological_models,
                    "rcm": rcm,
                    "gcm": gcm,
                    "ensemble_member": ens_member,
                    "period": period
                }   
                client.retrieve(dataset, request, file)
        
                # Unzip the file that was just downloaded, and remove the zip file
                with zipfile.ZipFile(file, 'r') as zObject:
                    zObject.extractall(path=data_folder_catch)
                os.remove(file)

We will make use of a preprocessing function to write model names and scenarios to the dataset dimensions:

In [15]:
def preprocess_flood_occurence(ds):
    filename = os.path.basename(ds.encoding['source'])
    _, _, _, _, gcm, scenario, _, rcm, _, time_period, _, _ = filename.split('_')
    return ds.expand_dims({
        'scenario': [scenario],
        'gcm_rcm': [f'{gcm}_{rcm}']
    }).assign_coords({
        'time_period': ('time', [time_period])
    })

We will read the dataset of extreme river discharges (absolute values) into a single dataset:

In [16]:
ds_flood = xr.open_mfdataset(
    os.path.join(data_folder_catch, 'rdisreturnmax*_tmean_abs_E-HYPEcatch*_catch_v1.nc'),
    preprocess=preprocess_flood_occurence
)
ds_flood

<xarray.Dataset> Size: 20MB
Dimensions:                (scenario: 3, gcm_rcm: 6, time: 4, id: 34810)
Coordinates:
  * scenario               (scenario) object 24B 'historical' 'rcp45' 'rcp85'
  * gcm_rcm                (gcm_rcm) object 48B 'ICHEC-EC-EARTH_CLMcom-CCLM4-...
  * id                     (id) int32 139kB 8801654 8000123 ... 9605711 9601936
  * time                   (time) datetime64[ns] 32B 1971-01-01 ... 2071-01-01
    time_period            (time) <U9 144B '1971-2000' ... '2071-2100'
Data variables:
    rdisreturnmax10_tmean  (scenario, gcm_rcm, time, id) float32 10MB dask.array<chunksize=(2, 1, 1, 34810), meta=np.ndarray>
    rdisreturnmax50_tmean  (scenario, gcm_rcm, time, id) float32 10MB dask.array<chunksize=(2, 1, 1, 34810), meta=np.ndarray>
Attributes: (12/29)
    CDI:                      Climate Data Interface version 1.8.2 (http://mp...
    Conventions:              CF-1.6
    NCO:                      netCDF Operators version 4.7.7 (Homepage = http...
    frequency:                year
    CDO:                      Climate Data Operators version 1.8.2 (http://mp...
    comment:                  -
    ...                       ...
    invar_hm_model_id:        Hydrological models in the order of the variabl...
    invar_experiment_name:    rcp45
    time_coverage_start:      19710101
    time_coverage_end:        20001231
    variable_name:            rdisreturnmax10_tmean
    contact:                  copernicus-support@ecmwf.int

This preprocessed dataset will be saved to disk for future use:

In [18]:
ds_flood.to_netcdf(os.path.join(data_folder, 'rdis_extremes_abs_E-HYPEcatch_allmodels.nc'))

The same procedure will be applied to the dataset of relative changes in extreme river discharges:

In [19]:
ds_flood_rel = xr.open_mfdataset(
    os.path.join(data_folder_catch, 'rdisreturnmax*_tmean_rel_E-HYPEcatch*_catch_v1.nc'),
    preprocess=preprocess_flood_occurence
)
ds_flood_rel

<xarray.Dataset> Size: 10MB
Dimensions:                (scenario: 2, gcm_rcm: 6, time: 3, id: 34810)
Coordinates:
  * scenario               (scenario) object 16B 'rcp45' 'rcp85'
  * gcm_rcm                (gcm_rcm) object 48B 'ICHEC-EC-EARTH_CLMcom-CCLM4-...
  * id                     (id) int32 139kB 8801654 8000123 ... 9605711 9601936
  * time                   (time) datetime64[ns] 24B 2011-01-01 ... 2071-01-01
    time_period            (time) <U9 108B '1971-2000' '1971-2000' '1971-2000'
Data variables:
    rdisreturnmax10_tmean  (scenario, gcm_rcm, time, id) float32 5MB dask.array<chunksize=(1, 1, 1, 34810), meta=np.ndarray>
    rdisreturnmax50_tmean  (scenario, gcm_rcm, time, id) float32 5MB dask.array<chunksize=(1, 1, 1, 34810), meta=np.ndarray>
Attributes: (12/30)
    CDI:                      Climate Data Interface version 1.8.2 (http://mp...
    Conventions:              CF-1.6
    NCO:                      netCDF Operators version 4.7.7 (Homepage = http...
    frequency:                year
    comment:                  -
    CDO:                      Climate Data Operators version 1.8.2 (http://mp...
    ...                       ...
    invar_experiment_name:    rcp45
    time_coverage_start:      20110101
    time_coverage_end:        20401231
    reference_period:         1971-2000
    variable_name:            rdisreturnmax10_tmean
    contact:                  copernicus-support@ecmwf.int

In [20]:
ds_flood_rel.to_netcdf(os.path.join(data_folder, 'rdis_extremes_rel_E-HYPEcatch_allmodels.nc'))

Now all of the data that we need for the analysis has been retrieved and aggregated. In the next notebooks, this data will be used to analyze the impact of climate scenarios on the seasonal and extreme river discharges as a proxy for flood hazard.

## Next step

Continue with the [hazard assessment using river discharge statistics](./hazard_assessment_discharge_analysis.ipynb).

## Contributors

Author of the workflow: Natalia Aleksandrova (Deltares)